# MODEL-FITTING OF EXTRAGALACTIC PLANETARY NEBULAE

This is an example of model-fitting with few observational constraints. Consider the planetary nebulae observed in the galaxy NGC 5128 by Walsh et al. (2012), for example objects F34 1, F34 2, F34 4, F34 7, and F34 11.

### For each object, try to fit an ionization bounded photoionization model to the observed data assuming a spherical geometry, constant density, a blackbody radiation, and comparing the observed line intensities with those given by the model. Before starting, think of the policy you will follow to find your best model. To analyze the goodness of your fit, use first a khi-sqare method on the line intensities, with weights inversely proportional to the line intensities. This is a widely used method.

##### We import some libraries.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pyneb as pn
import pyCloudy as pc
from pyCloudy.utils.misc import convert_label, sextract
pc.config.cloudy_exe = 'cloudy.exe'
models_dir = '../../Models/'

In [ ]:
emis_file = '../Data/NGC5128_56_12b_emis.dat'

In [ ]:
class In(object):

    def __init__(self, model_dir, name, r_in, dens, Teff, Q0, abund_dict, distance, grains=None):
        """
        Defining the parameters of the model.
        """
        # combining dir and name
        self.model_name = '{0}/{1}'.format(model_dir, name)
        # set the input parameters to self. variables
        self.r_in = r_in
        self.dens = dens
        self.distance = distance
        self.abund_dict = abund_dict
        self.Q0 = Q0
        self.Teff = Teff
        self.grains = grains
        # define more options to Cloudy
        self.options = ('no molecules',
                        'no level2 lines',
                        'no fine opacities',
                        'atom h-like levels small',
                        'atom he-like levels small',
                        'COSMIC RAY BACKGROUND',
                        'element limit off -8',
                        )

        
    def print_model(self):
        """
        Preparing and printing the Cloudy input file
        """
        # define the name of the model
        model = pc.CloudyInput(self.model_name)
        # send the variables to the CloudyInput object to be printed 
        model.set_radius(self.r_in)
        model.set_cste_density(self.dens)
        model.set_distance(self.distance, unit='Mpc')
        model.set_abund(ab_dict = self.abund_dict)
        model.set_grains(self.grains)
        model.set_BB(Teff=self.Teff, lumi_unit='q(H)', lumi_value=self.Q0)
        # this is the file containing the list of emissivities we want
        model.read_emis_file(emis_file)
        model.set_iterate(0)
        model.set_sphere()
        model.set_other(self.options)
        # print the input file
        model.print_input(to_file = True, verbose = False)
        # store the model in a self variable to further interactions if needed
        self.model = model
        
    def run_model(self):
        # call the Cloudy runner
        self.model.run_cloudy()


In [ ]:
class Outs(object):
    
    def __init__(self, model_dir, models, emis_file):
        """
        Object dealing with the Cloudy outputs.
        Usage: Mouts = Outs(models_dir, 'M_10_D')
        """
        self.Ms = pc.load_models('{0}/{1}'.format(model_dir, models))
        self.read_obs(emis_file)
        
    def read_obs(self, emis_file):
        """
        Read the observation file pointed by the emis_file variable.
        This function is called by __init__
        The line intensities are corrected from extinction using Ha/Hb = 2.85
        """
        self.obs_txt = np.genfromtxt(emis_file, dtype=["a13","float", "float"], 
                            delimiter=[13,8, 6], names = ['label', 'i_obs', 'e_obs'], usecols = (0, 1, 2))

        # redenning correction
        Hb = self.obs_txt['i_obs'][self.obs_txt['label'] == 'H  1 4861.36A']
        Ha = self.obs_txt['i_obs'][self.obs_txt['label'] == 'H  1 6562.85A']
        self.RC = pn.RedCorr(law = 'F99')
        self.RC.setCorr(Ha / Hb / 2.85, 6563, 4861)
        for line in self.obs_txt:
            lambda_ = np.float(line['label'][-8:-1])
            line['i_obs'] *= self.RC.getCorrHb(lambda_)
        
    def get_i_obs(self, label):
        """
        return the line intensity given the line label
        """
        i_label = (self.obs_txt['label'] == label)
        if i_label.sum() != 1:
            return None
        else:
            return self.obs_txt[i_label]['i_obs'][0]
        
    def pretty_print(self, str1, list1):
        """
        pretty print the values of a list.
        """
        if type(list1[0]) == type(''):
            print('{0:32s}'.format(str1) + ' '.join(['{0:>9}'.format(i) for i in list1]))
        else:
            print('{0:32s}'.format(str1) + ' '.join(['{0:>9.3f}'.format(i) for i in list1]))

    def print_ratio(self, label1, label2, title):
        """
        pretty print the ratio of emission lines
        """
        ref_pycloudy1 = convert_label(label1)
        ref_pycloudy2 = convert_label(label2)
        obs_ratio = self.get_i_obs(label1) / self.get_i_obs(label2)
        mod_ratio = [M.get_emis_vol(ref_pycloudy1) / M.get_emis_vol(ref_pycloudy2) for M in self.Ms]
        str1 = '{0:12s}   {1:>8.3f}'.format(title, obs_ratio)
        self.pretty_print(str1, mod_ratio)

    def print_res(self):
        """
        Print the results of all the models.
        """
        Hbeta_red = -16.170
        names = [M.model_name_s for M in self.Ms]
        self.pretty_print('MODEL', names)
        r_in = [np.log10(M.r_in) for M in self.Ms]
        self.pretty_print('Inner radius', r_in)
        r_out = [np.log10(M.r_out) for M in self.Ms]
        self.pretty_print('Outer radius', r_out)
        Teff = [np.float(sextract(M.out['Blackbody'], 'Blackbody ', ' '))/1e3 for M in self.Ms]
        self.pretty_print('Effective Temp kK', Teff)
        dens = [np.log10(M.nH[0]) for M in self.Ms]
        self.pretty_print('Hydrogen density', dens)
        Q0 = [np.log10(M.Q0) for M in self.Ms]
        self.pretty_print('Q0', Q0)
        logUmean = [M.log_U_mean for M in self.Ms]
        self.pretty_print('<logU>', logUmean)
        abHe = [M.abund['He'] for M in self.Ms]
        self.pretty_print('He/H', abHe)
        abC = [M.abund['C'] for M in self.Ms]
        self.pretty_print('C/H', abC)
        abN = [M.abund['N'] for M in self.Ms]
        self.pretty_print('N/H', abN)
        abO = [M.abund['O'] for M in self.Ms]
        self.pretty_print('O/H', abO)
        abNe = [M.abund['Ne'] for M in self.Ms]
        self.pretty_print('Ne/H', abNe)
        abS = [M.abund['S'] for M in self.Ms]
        self.pretty_print('S/H', abS)
        abAr = [M.abund['Ar'] for M in self.Ms]
        self.pretty_print('Ar/H', abAr)
        Hb = [np.log10(M.get_emis_vol('H__1_486136A')/self.RC.getCorr(4861)/
                       (4.*np.pi*(M.distance*pc.CST.KPC)**2)) for M in self.Ms]
        self.pretty_print('Hbeta           {:.2f}'.format(Hbeta_red+np.log10(self.RC.getCorr(4861.))), Hb)
        
        for line in self.obs_txt:
            ref_pycloudy = convert_label(line['label'])
            try:
                mod_intens = [M.get_emis_vol(ref_pycloudy) / M.get_emis_vol('H__1_486136A') * 100 for M in self.Ms]
                str1 = '{0} {1:>7.1f} +/-{2:>3.0f} '.format(line['label'], line['i_obs'], line['e_obs'])
                self.pretty_print(str1, mod_intens)
            except:
                print('Something wrong with {0}'.format(line['label']))
        try:
            self.print_ratio('S  2  6730.82A', 'S  2  6716.44A', 'Dens(SII)')
        except:
            pass
        try:
            self.print_ratio('BLND 3727.00A', 'O  3 5006.84A', 'OII/III')
        except:
            pass


In [ ]:
pc.log_.level = 3
# define the model name and properties
model_name = 'M10_A'
# define the ind
i = 1

r_in = 15. 
dens = 4.
Teff = 40000 
Q0 = 46
distance = 3.0
ab_dict = {'He':-1.00, 'C':-3.60, 'N':-3.45, 'O':-3.95 , 'Ne':-4.8, 'Mg':-4.95,
           'Si':-4.90, 'S':-5.35, 'Cl':-7.00, 'Ar':-5.7, 'Fe':-7.40}

pc.log_.level = 3
# create the object that generates the input files
Min = In(models_dir, '{0}_{1}'.format(model_name, i), r_in, dens, Teff, Q0,
         ab_dict, distance)
Min.print_model()

# run the models
Min.run_model()

# read the models
pc.log_.level = 2
Mouts = Outs(models_dir, model_name, emis_file)
# output the parameters and line intensities, with the observations in 1rst column
Mouts.print_res()


In [ ]:
# the following is to have the nice style in the Notebook.
# Don't remove this.
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()